In [1]:
# 1. Library imports
import pandas as pd
from IPython.display import Markdown, display
from extraction import llmAgent
from ppi_deprescribe import merge_results, ppi_deprescribe
import os 
from pathlib import Path 


f:\LangChain\DataSci210_MedicationDeprescriber\src\extraction.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
f:\LangChain\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

path = Path('F:/LangChain/data')
groq = os.environ['groqkey']
pd.set_option('display.max_colwidth', None)

In [3]:

def deprescribe(key):
    # extract information
    llm_agent = llmAgent(groq_key=groq, data_path=path)

    results_dict = {
        "diagnosis_dict": llm_agent.extract_diagnosis(encounter_key=key),
        "encounter_dict": llm_agent.extract_encounter_info(encounter_key=key),
        # Is the reasoning in the json or sepearte?
        # Should the reasoning be included in any of them or just the diangosis with the reasoning seperate?
        "notes_dict": llm_agent.extract_notes(encounter_key=key),
    }
    print(results_dict['notes_dict'])
    # # #   master formatter step   # # #
    # merge the diagnosis booleans (just use OR logic for now)
    # make a final "reasoning" behind the recommendation
    final_dict = merge_results(results_dict=results_dict)

    # feed the three reasonings to LLM to get a single summary
    final_reasoning = llm_agent.summarize_reasonings(results_dict=results_dict)

    # # #   get recommendation from PPI algorithm   # # #
    recommendation_str = ppi_deprescribe(patient_diagnosis=final_dict)
    return recommendation_str, final_reasoning
    

In [4]:
labeled_data = pd.read_csv('F:/LangChain/data/LabeledResponses.csv')
labeled_data = labeled_data.head(1)
labeled_data.head()

,key,reason,recommendation,GS_response,Note
0,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN


In [5]:
def apply_deprescribe(row):
    try:
        recommendation_str, final_reasoning = deprescribe(row['key'])
        return pd.Series([recommendation_str, final_reasoning])
    except:
        return pd.Series([None, None])

labeled_data[['Rec_Returned', 'Reasoning']] = labeled_data.apply(apply_deprescribe, axis=1)

{'Mild to moderate esophagitis': False, 'GERD': False, 'Peptic Ulcer Disease': False, 'Upper GI symptoms': False, 'ICU Stress Ulcer Prophylaxis': False, 'Barretts Esophagus': False, 'Chronic NSAID use with bleeding risk': False, 'Severe esophagitis': False, 'Documented history of bleeding GI ulcer': True, 'H pylori infection': True, 'Reasoning': "The patient has a history of gastrointestinal bleeding and has been diagnosed with H. pylori infection. The patient's recent labs show a low hemoglobin level and a positive H. pylori IgG antibody test. The patient's endoscopy results show a small AVM in the stomach and erosions in the duodenum. The patient's symptoms and test results suggest a high risk for gastrointestinal bleeding."}


In [6]:
labeled_data.head()

,key,reason,recommendation,GS_response,Note,Rec_Returned,Reasoning
0,D6253A5CE371EA,example with gastrointestinal bleed should continue PPI,continue,"This patient has a history of a gastrointestinal bleeding which was present on the admission diagnosis. The pantoprazole 20mg taken twice daily should be continued. The GI bleed is mentioned in the note from 09/14/2023, on 09/17/2023, and on 09/20/2023.",NaN,continue,"The patient has a documented history of bleeding GI ulcer, as indicated by a primary diagnosis of gastrointestinal hemorrhage, history of gastrointestinal bleeding, and presence of upper GI symptoms. The patient's lab results and endoscopy findings also support this diagnosis. There is no specific information provided about the other conditions, so they are marked as false, except for H. pylori infection in one case, which is confirmed by lab results and endoscopy findings."


In [7]:
labeled_data.to_csv('F:/LangChain/data/LabeledResponses_ReturnedResponse.csv', index=False)